# Focused Ion Beam Molecular Dynamics (fibmd) Tool

In [61]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from string import Template
import moviepy.editor as mpy
import hublib.use
from hublib.cmd import runCommand

In [62]:
%use openmpi-1.6.3-gnu-4.7.2

# Creating widgets for critical input

In [63]:
title = widgets.Label(value="FIBMD Tool", width=10)

keVin = widgets.BoundedFloatText(
    description='Ion Energy',
    value=2000,
    min=1000,
    max=60000,
    step=100
)

dimlab = widgets.Label(value="Target dimensions (in Si unit cells)")

xdimin = widgets.BoundedIntText(
    value=4,
    min=4,
    max=40,
    step=1,
    description='X:'
)

ydimin = widgets.BoundedIntText(
    value=4,
    min=4,
    max=40,
    step=1,
    description='Y: '
)

zdimin = widgets.BoundedIntText(
    value=12,
    min=4,
    max=40,
    step=1,
    description='Z:'
)

dimensions = widgets.HBox([xdimin, ydimin, zdimin])

pin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 10,
    step = 1
)
procsin = widgets.HBox([widgets.Label("# of Processors:"),pin])

nin = widgets.BoundedIntText(
    value = 1,
    min = 1,
    max = 100,
    step = 1
)
nljin = widgets.HBox([widgets.Label("# Ions to fire:"),nin])

ain  = widgets.BoundedIntText(
    value = 500,
    min = 0,
    max = 1e6,
    step = 100
)
atmoutin = widgets.HBox([widgets.Label(".xyz output frequency"),ain])

din = widgets.FloatText(
    value = 1,
    step = 0.1
)
dtin = widgets.HBox([widgets.Label("Time between ion impacts in ps:"),din])

fin = widgets.FloatText(
    value = 2
)
fwhmin = widgets.HBox([widgets.Label("FWHM of ion beam in nm:"),fin])
runbut = widgets.Button(description='Run Simulation')


In [64]:
widgets.VBox([title,keVin,dimlab,dimensions,procsin,nljin,atmoutin,dtin,fwhmin,runbut])

VBox(children=(Label(value='FIBMD Tool'), BoundedFloatText(value=2000.0, description='Ion Energy', max=60000.0, min=1000.0, step=100.0), Label(value='Target dimensions (in Si unit cells)'), HBox(children=(BoundedIntText(value=4, description='X:', max=40, min=4), BoundedIntText(value=4, description='Y: ', max=40, min=4), BoundedIntText(value=12, description='Z:', max=40, min=4))), HBox(children=(Label(value='# of Processors:'), BoundedIntText(value=1, max=10, min=1))), HBox(children=(Label(value='# Ions to fire:'), BoundedIntText(value=1, min=1))), HBox(children=(Label(value='.xyz output frequency'), BoundedIntText(value=500, max=1000000, step=100))), HBox(children=(Label(value='Time between ion impacts in ps:'), FloatText(value=1.0, step=0.1))), HBox(children=(Label(value='FWHM of ion beam in nm:'), FloatText(value=2.0))), Button(description='Run Simulation', style=ButtonStyle())))

In [65]:
def get_template():
    if ain.value == 0:
        atmoutin1 = -1
        atmoutin2 = -1
    else:
        atmoutin1 = int(ain.value/10)
        atmoutin2 = ain.value
    inputs = list([str(keVin.value),str(xdimin.value),str(ydimin.value),str(zdimin.value),
                         str(pin.value),str(nin.value),str(atmoutin1),str(atmoutin2),
                         str(din.value*1e-12),str(fin.value)])
    tags = list(['keV','xdim','ydim','zdim',
                 'procs','nlj','atmout1','atmout2',
                 'dtion','fwhm'])

    input_dict = dict(zip(tags, inputs))
    temp_contents = open('sige.in.template').read()
    tempstr = Template(temp_contents)
    print(tempstr)
    # Python template strings replace "${identifier}" with the value.
    # If substituting more than one value, use a dictionary.
    return tempstr.substitute(input_dict)

In [66]:
def run_mdrun2():
    with open('sige.in', 'w') as tfile:
        tfile.write(get_template())
    res, stdout, stderr = runCommand('mpirun -np %i src/mdrun2'%(pin.value))
    #!mpirun -np $procsin.value src/mdrun2 #> out.log | tail -10 out.log #> out.log# -10 out.log
    return

In [67]:
# This is the main function called when the Run button is clicked.
def run_simulation(ignore):
    # change label on Button
    runbut.disabled=True
    runbut.description='Running'

    run_mdrun2()

    # Change button label back and re-enable
    runbut.description='Run Simulation'
    runbut.disabled=False
    

In [68]:
runbut.on_click(run_simulation)